In [ ]:
import os
import sys
import re
import json                                       
import gnssrefl.gps as g
import gnssrefl.rinex2snr as rnx
import gnssrefl.quickLook_function as quick
import gnssrefl.gnssir as guts
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns; sns.set_theme(style="whitegrid");

from datetime import datetime

bin_path = os.path.abspath(os.path.join('../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)
    
import check_parameters 
import environment

%matplotlib inline

In [ ]:
#Making sure environment variables are set
exists = environment.check_environment()
if exists == False:
    environment.set_environment()
else:
     print('environment variable ORBITS path is', os.environ['ORBITS'],
          '\nenvironment variable REFL_CODE path is', os.environ['REFL_CODE'],
          '\nenvironment variable EXE path is', os.environ['EXE'])
        
# import the crx2rnx file
environment.download_crx2rnx()

In [ ]:
### Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



In [ ]:
### Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



In [ ]:
### Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



### Ross Ice Shelf, Antarctica

**Station Name:**  lorg

**Location:**  Lorne, Antarctica

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/ACF6-YT23](https://doi.org/10.7283/ACF6-YT23)

**Ellipsoidal Coordinates:**

- Latitude:  -78.18365
- Longitude: 170.03361
- Height:  	-7.778 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/lorg)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LORG.sta)

[Google Maps Link](https://goo.gl/maps/bSAuLXLLMmzWqPdW9) 

<img src="https://gnss-reflections.org/static/images/LORG.jpg" width="500">



### Data Summary

Station lorg is on the Ross Ice Shelf, Antarctica.
The site is a largely featureless ice plain with no obstructions (see photo above). 
The site was installed on November 27, 2018 and decommissioned and removed on November 15, 2019. 
It recorded only GPS frequencies during its operation. 

LORG is an example station on the [GNSS-IR web app.](https://gnss-reflections.org/api?example=lorg) 

There are no significant topographic features near the station, so it is recommended to use default values 
for the elevation mask. An azimuth mask is not required.

If you are curious about the position time series for the antenna, you can use *download_unr lorg* for more information.



In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=lorg" width="1000" height="900"></iframe>

## Take a look at the Data

First make a SNR file.

In [ ]:
station = 'lorg'
year = 2019 
doy = 205

lat = -78.1836
long = 170.0336
height = -7.722

In [ ]:
# Can view the parameters here
#check_parameters.rinex2snr?

In [ ]:
args = check_parameters.rinex2snr(station,year,doy, translator='hybrid')
rnx.run_rinex2snr(**args)



Use **quickLook** to produce a periodogram similar to the one in the web app [(For details on quickLook output)](../../docs/quickLook_desc.md). quickLook is set to use the L1 frequency by default:

In [ ]:
# making a plotting function for the quicklook function
def quicklook_results(args, values):
    freq = {1:'L1', 20: 'L2C', 5:'L5'}
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
    quadrants = ['NW', 'NE', 'SW', 'SE']
    axes = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

    for i, quadrant in enumerate(quadrants):
        satellites = values[quadrant].keys()
        fail_satellites = values[f'f{quadrant}'].keys()

        for failsat in fail_satellites:
            axes[i].plot(values[f'f{quadrant}'][failsat][0], values[f'f{quadrant}'][failsat][1], color='lightgrey') 
        for sat in satellites:
            axes[i].plot(values[quadrant][sat][0], values[quadrant][sat][1])

    ax[0,0].set_title('Northwest', size=14)
    ax[0,1].set_title('Northeast',size=14)
    ax[1,0].set_title('Southwest', size=14)
    ax[1,1].set_title('Southeast', size=14)

    for ax in axes:
        ax.set_xlabel('reflector height (m)', size=14)
        ax.set_ylabel('volts/volts', size=14)
        ax.grid()
    
    fig.suptitle(f'GNSS Station {args["station"].upper()}, {args["year"]} doy {args["doy"]}, freq {freq[args["f"]]}, elevation angles {args["e1"]}-{args["e2"]} \n', size=16)
    fig.tight_layout()
    plt.show()
    
    
def quicklook_metrics(args, values):
#     fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
    quadrants = ['NW', 'NE', 'SW', 'SE']
    
    # re-organizing the data in a plotting friendly format
    success_data = {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise':[], 'Amplitude': []}
    fail_data =  {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise': [], 'Amplitude': []}
    
    for i, quadrant in enumerate(quadrants):
        for j in values[quadrant].keys():
            success_data['Azimuth'].append(datakeys[quadrant][j][0])
            success_data['Reflector Height'].append(datakeys[quadrant][j][1])
            success_data['Peak to Noise'].append(datakeys[quadrant][j][5])
            success_data['Amplitude'].append(datakeys[quadrant][j][4])
        for k in values[f'f{quadrant}'].keys():
            fail_data['Azimuth'].append(datakeys[f'f{quadrant}'][k][0])
            fail_data['Reflector Height'].append(datakeys[f'f{quadrant}'][k][1])
            fail_data['Peak to Noise'].append(datakeys[f'f{quadrant}'][k][5])
            fail_data['Amplitude'].append(datakeys[f'f{quadrant}'][k][4])

    return pd.DataFrame(success_data), pd.DataFrame(fail_data)

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

Compare the periodograms for other frequencies: L2C and L5. They should be similar to the L1 periodogram, except that there
will be fewer satellite traces because only GPS satellites launched after 2005
broadcast L2C and only satellites after 2010 broadcast L5.
The northwest qudarant is the noisiest and one could certainly try to improve the results by restricting some azimuths there.
This is for L2C:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=20)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

This is for L5:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=5)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

### Analyze the Data
Now prepare to analyze the data using **gnssir**.  First you need to create a set of analysis instructions.
The default settings only need the station name, latitude, longitude, and ellipsoidal height.

In [ ]:
# Here we can see what parameters are available and what the defaults are:
#check_parameters.make_json?

In [ ]:
# we saved the lat, long, and height earlier
check_parameters.make_json(station, lat, long, height)

In [ ]:
# This is the json file that was created
json_file = 'input/lorg.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Next make some snr files for a time span of about eight months. Restrict the search to the UNAVCO archive to make the
code run faster (otherwise it will check three other archives as well). The resulting SNR files will be stored in $REFL_CODE/2019/snr/lorg.

In [ ]:
#check_parameters.rinex2snr?

In [ ]:
args = check_parameters.rinex2snr(station, year=2019, doy=1, doy_end=233, archive='unavco', translator='fortran')
rnx.run_rinex2snr(**args)

Run **gnssir** for all the SNR files from **rinex2snr**.

In [ ]:
#check_parameters.gnssir?

In [ ]:
doy = 1
doy_end=233
args = check_parameters.gnssir(station, year, doy=doy, doy_end=doy_end, screenstats=False)

year_list = list(range(year, args['year_end'] + 1))
doy_list = list(range(doy, args['doy_end'] + 1))
for year in year_list:
    args['args']['year'] = year
    for doy in doy_list:
        args['args']['doy'] = doy
        guts.gnssir_guts(**args['args'])

The default does not send any plots to the screen. If you do want to see them, set -plt True:

In [ ]:
doy = 1
args = check_parameters.gnssir(station, year, doy=1, plt=True, screenstats=False)
year_list = list(range(year, args['year_end'] + 1))
doy_list = list(range(doy, args['doy_end'] + 1))
print(doy_list)
for year in year_list:
    args['args']['year'] = year
    for doy in doy_list:
        args['args']['doy'] = doy
        guts.gnssir_guts(**args['args'])

Unlike **quickLook**, **gnssir** plots the periodograms for each frequency with no quadrants. To see the next frequency, close the current plot.

These results can be cleaned up by eliminating various azimuths and requiring stronger peaks in the periodograms in the json file, but for now just use defaults.
The reflector heights are stored in $REFL_CODE/2019/results/lorg.

The **daily_avg** command will calculate the daily average reflector height from the daily output files. To minimize outliers in these daily averages, a median filter is set to 0.25 meters and the required minimum number of daily satellite tracks is set to 50.

In [ ]:
check_parameters.daily_avg?

In [ ]:
check_parameters.daily_avg(station, medfilter=.25, ReqTracks=50)

This shows the range of reflector heights each day, with each day represented by a different color.

There are also optional inputs for saving a text file of the daily averages. 
The plot is stored at $REFL_CODE/Files/lorg_RH.png 
This is not yet perfect - there are some outliers which have been **circled in red**. 
But it is an excellent example of how easy it is to use the default gnssrefl code to estimate snow accumulation in Antarctica.

In this exercise L1, L2C, and L5 signals were used (i.e. only GPS data). The reflector heights give information 
about snow accumulation at lorg. Notice that when reflector heights are plotted, the y-axis is reversed so that the reflector height gets smaller as the snow layer increases and the distance between the antenna and snow surface decreases.  